In [16]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import datasets
import utils
import shutil
import os

In [17]:
root_path = "./data/tdsc/tumors_3d"
transpose_shapes = [((0,1,2), 'z'), ((2,0,1), 'y'), ((1,2,0), 'x')]
data = datasets.TDSC()
print(len(data))

100


In [18]:
def find_tumors(mask: np) -> np:
    '''
    Finds all bounding boxes in volume containing tumors
    
    parameters:
    - mask: numpy array with shape (x,y,d)
    
    returns:
    - numpy array with (None, [x,y,w,h,d])
    '''
    
    _, _, depth = mask.shape
    bbxs = []
    for d in range(depth):
        sli = mask[:,:,d]
        if sli.max() != 0:
            *bbx, = utils.image.find_bbx(sli)
            bbx.append(d)
            bbxs += [bbx]
    return np.array(bbxs)

In [19]:
def find_largest_bounding_box(bbxs: list):
    
    xl,yl = 1000, 1000
    xh,yh = 0, 0
    
    for bbx in bbxs:
        
        x,y,w,h,_ = bbx
        xl = min(xl, x)
        yl = min(yl, y)
        xh = max(xh, x+w)
        yh = max(yh, y+h)
        
    return (xl,yl), (xh,yh)

In [20]:
for shape, cat in transpose_shapes:    
    for index, sample in enumerate(data):
        
        v,m,y = sample
        v = np.transpose(v, shape)
        m = np.transpose(m, shape)
        bbxs = find_tumors(m)
        l,h = find_largest_bounding_box(bbxs)
        tumor = np.zeros([h[1]-l[1], h[0]-l[0], len(bbxs)])
        mask = np.zeros([h[1]-l[1], h[0]-l[0], len(bbxs)])
        
        for idx, bbx in enumerate(bbxs):    
            _, _, _, _, d = bbx
            tumor[:,:,idx] = v[l[1]:h[1], l[0]:h[0], d]
            mask[:,:,idx] = m[l[1]:h[1], l[0]:h[0], d]
            
        np.save(f"./data/tdsc/tumors_3d/{cat}/data/{y}/{index}", tumor)
        np.save(f"./data/tdsc/tumors_3d/{cat}/mask/{y}/{index}", mask)